
**Atividade - Inferência de Dados**

<br>

**Nomes e RGMs:**

Gabriela Cristina Schmitt   - RGM 25733150, 

Felipe dos Santos Fabienski - RGM 13112368,

Jennifer Mayara de Paiva Goberski, 25810723

<br>


Considerando que o modelo de clusters treinado já está aberto em memória:

Receba (ou simule) os dados de uma nova instância. Lembre-se de que esses dados estarão no formato "natural" da instância
Normalize a instância utilizando o normalizador numérico obtido e salvo durante a fase de treinamento do modelo atual
O resultado da nova instância normalizada deve ser convertido para o mesmo formato (mesmas colunas) do arquivo normalizado e utilizado para o treinamento do modelo atual
Após essa equalização, utilize o método predict() do modelo atual, de maneira a determinar com qual grupo/cluster essa nova istância é similar.
Mostre os dados desnormalizados relativo ao cluster ao qual a nova instância pertence.


# Documentação das funções
*1. load_new_df:*
Carrega os dados contidos da nova instânci localizado no path informado no construtor da classe Modelo e os salva no atributo **data**;

*2. get_normalized_numDF:*
Normaliza os dados numéricos contidos no atributo **data**, utilizando um normalizador previamente criado, e os salva no atributo **Norm_num**;

*3. get_normalized_catDF:*
Normaliza os dados categóricos contidos no atributo **data**, utilizando um normalizador previamente criado, e os salva no atributo **Norm_cat**;

*4. get_normalized_df:*
Junta os dados numéricos e categóricos e os une em um DataFrame, que é salvo no atributo **NormData**;

*5. predict_df:*
Usando um modelo de cluster previamente treinado, é chamada a função predict para determinar a qual grupo os novos dados pertencem. Essa predição é salva no atributo **PredictResult**;

*6. get_denorm_df:*
Nessa função os dados normalizados do modelo de cluester são desnormalizados para que possam ser usados na próxima função e possam ser exibidos ao usuário. Esses dados são salvos no atributo **NormResultDf**;

*7. get_result_by_index:*
Essa função pegas os índices dos centróides do modelo de cluester apontados pelo predict e os exibe para o usuário de forma desnormalizada;

*8. process_data:*
Função chamada pelo usuário após instanciar um objeto da classe Modelo, ela executa as funções acima na mesma ordem que foram descritas.

In [ ]:
!pip install pandas==1.5.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

from sklearn import preprocessing
import pickle 
import pandas as pd 
import numpy as np

class Modelo:
  def __init__(self, path): 
    self.path = path 
    self.data = None
    self.Norm_num = None
    self.Norm_cat = None
    self.NormData = None
    self.NormResultDf = None
    self.PredictResult = None
    self.dummy_separator = '&'
    self.normalizer = pickle.load(open('/content/drive/MyDrive/Sistemas Inteligentes Avancados/normalizador1.model', 'rb'))    
    self.cluster_model = pickle.load(open('/content/drive/MyDrive/Sistemas Inteligentes Avancados//cluster1.model', 'rb')) 

  def __load_new_df(self): 
    try:
      dados = pd.read_csv(self.path, sep = ';')
      self.data = dados
    except:
      raise ValueError("Unable to load data from file: " + self.path)

  def __get_normalized_numDF(self):
    if self.data is None: 
      raise ValueError("No data loaded yet. Please call load_data() first.")

    else: 
      dados = self.data

      dados_num = dados.drop(columns=['sexo'])
      modelo_normalizador = self.normalizer.fit(dados_num)

      # Aplicando o normalizador na nossa base 
      dados_num_normalizado = self.normalizer.fit_transform(dados_num)   

      # Tranformando array em DF
      self.Norm_num = pd.DataFrame(dados_num_normalizado, columns = dados_num.columns) # antipattern 


  def __get_normalized_catDF(self):
      if self.data is None: 
        raise ValueError("No data loaded yet. Please call load_data() first.")

      else: 
        dados_cat = self.data[['sexo']]
        dados_cat_normalizado = pd.get_dummies(dados_cat, prefix_sep=self.dummy_separator) 
        self.Norm_cat = dados_cat_normalizado

  def __get_normalized_df(self): 
    self.__get_normalized_numDF()
    self.__get_normalized_catDF()
    self.NormData = self.Norm_num.join(self.Norm_cat, how = 'left')
    print('\n -------------DADOS NORMALIZADOS-------------------\n')
    print(self.NormData)

  def __predict_df(self):
    self.PredictResult = self.cluster_model.predict(self.NormData)
    print('\n -------------RESULTADOS PREDICT-------------------\n')
    print(self.PredictResult)
    

  def __get_denorm_df(self):
    cluster = pd.DataFrame(self.cluster_model.cluster_centers_, columns=self.NormData.columns)
    
    cluster_data_num = cluster.drop(columns=self.Norm_cat.columns)
    denorm_num = self.normalizer.inverse_transform(cluster_data_num)
    denorm_num = pd.DataFrame(denorm_num, columns=cluster_data_num.columns)

    cluster_cat_data = cluster[self.Norm_cat.columns].round(0).abs().astype(int)==1
    denorm_cat = pd.from_dummies(cluster_cat_data)


    dummy_separator2 = '\n'
    denorm_cat_array = []

    for linha in range(0,len(denorm_cat)): 
      denorm_cat_array.append( str( str(denorm_cat.iloc[linha]).split(self.dummy_separator)[1]).split(dummy_separator2)[0])

    denorm_cat = pd.DataFrame(denorm_cat_array, columns=[str(denorm_cat.iloc[1]).split(self.dummy_separator)[0].strip()])

    self.NormResultDf = denorm_num.join(denorm_cat, how = 'left')
    print('\n -----------CENTROIDES DO CLUSTER------------------\n')
    print(self.NormResultDf)

  def __get_result_by_index(self):

    print('\n -------------------RESULTADOS---------------------\nENTRADAS:')
    [print(line, '\n') for line in self.data.iterrows()]
    summary = lambda data, index: [print(str(data.loc[predict]) + '\n') for predict in index]
    print('\nCLUSTERS RESPECTIVOS:')
    summary(self.NormResultDf, self.PredictResult)


  def process_data(self):
    self.__load_new_df()
    self.__get_normalized_df()
    self.__predict_df()
    self.__get_denorm_df()
    self.__get_result_by_index()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
caminho = '/content/drive/MyDrive/Sistemas Inteligentes Avancados/inferencia.csv'
modelo1 = Modelo(caminho)
modelo1.process_data()


 -------------DADOS NORMALIZADOS-------------------

   idade  altura  Peso  sexo&F  sexo&M
0    1.0     0.0   0.0       1       0
1    0.0     1.0   1.0       0       1

 -------------RESULTADOS PREDICT-------------------

[4 6]

 -----------CENTROIDES DO CLUSTER------------------

        idade  altura       Peso sexo
0   13.283951     1.0  87.791383    F
1   13.681481     1.0  77.970068    M
2    6.567988     2.0  69.218210    F
3   12.757576     2.0  88.756463    M
4    9.363636     1.0  70.626442    F
5    7.547581     1.0  67.441461    M
6    5.775253     2.0  84.170068    M
7   15.435017     2.0  74.280272    F
8    8.143939     2.0  89.265306    F
9    5.763015     1.0  92.251177    M
10   6.723906     1.0  91.074830    F
11  14.222222     2.0  69.119534    M

 -------------------RESULTADOS---------------------
ENTRADAS:
(0, idade     17
altura     1
Peso      60
sexo       F
Name: 0, dtype: object) 

(1, idade       3
altura      2
Peso      100
sexo        M
Name: 1, dtype: 